## Visualizando mapa

In [1]:
# Adiciona bibliotecas de visualização
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

output_notebook()

Loading BokehJS ...

In [2]:
# Marcador de coordenadas
SP = x_range,y_range = ((-5247500,-5135550), (-289990,-5123549))

p = figure(tools='pan, wheel_zoom', x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator", plot_width=960)

In [3]:
# Define o tipo de mapa
# url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
# url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager_labels_under/{Z}/{X}/{Y}.png'
# url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager_no_labels_no_buildings/{Z}/{X}/{Y}.png'
# url = 'http://a.basemaps.cartocdn.com/rastertiles/light_all/{Z}/{X}/{Y}.png'
url = 'http://a.basemaps.cartocdn.com/rastertiles/dark_all/{Z}/{X}/{Y}.png'
# url = 'http://a.basemaps.cartocdn.com/rastertiles/dark_nolabels/{Z}/{X}/{Y}.png'

attribution = "Mapa da cidade de São Paulo"

p.add_tile(WMTSTileSource(url=url, attribution=attribution))

TileRenderer(id='1035', ...)

In [4]:
def wgs84_to_web_mercator2(df, lon='LONGITUDE', lat='LATITUDE'):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    #return df

In [5]:
furto = ['Furto (art. 155)', 
         'Furto de coisa comum (art. 156)',
         'Furto qualificado (art. 155, §4o.)']

roubo = ['Roubo (art. 157)']

homicidio = ['Homicídio culposo (art. 121, §3o.)',
             'Homicídio culposo/veículo(Art. 302)',
             'Homicídio qualificado (art. 121, §2o.)',
             'Homicídio simples (art. 121)']

lesao_corporal = ['A.I.-Lesão corporal (art. 129)',
                  'Lesão corp GRAVÍSSIMA (art. 129, §2o.)',
                  'Lesão corporal (art 129 § 9º)',
                  'Lesão corporal (art. 129)',
                  'Lesão corporal culposa (art. 129. §6o.)',
                  'Lesão corp culposa/veículo(Art. 303)',
                  'Lesão corporal GRAVE (art. 129, §1o.)',
                  'Lesão corporal seguida de morte (art. 129, §3o.)']

estupro = ['A.I.-Estupro de vulneravel (art.217-A)',
           'Estupro (art.213)',
           'Estupro de vulneravel (art.217-A)']

trafico = ['Drogas (Art.33, caput)',
           'Induzir uso de droga(Art.33,§2º)',
           'Oferecer droga(Art.33,§3º)',
           'Porte de entorpecente (Art. 16)',
           'Porte de objeto destinado à fabricação de entorpecente (Art. 13)',
           'Tráfico de entorpecente (Art. 12)']

In [6]:
def crime_classification(df):
    """Generalizes the type of crime"""
#     print(df)
    if df in furto:
        return 'Yellow'
    elif df in roubo:
        return 'Orange'
    elif df in homicidio:
        return 'DeepSkyBlue'
    elif df in lesao_corporal:
        return 'YellowGreen'
    elif df in estupro:
        return 'Red'
    elif df in trafico:
        return 'SpringGreen'
    else:
        return 'White'

In [7]:
def read_data(year):
    return pd.read_csv('./crime-data-in-brazil/BO_'+ year +'.csv')

In [8]:
#ds_2015 = read_data('2015')
ds_2016 = read_data('2016')

/Users/thiagolima/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (19,21) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [9]:
n_2016 = ds_2016[['ANO', 'MES', 'RUBRICA', 'LATITUDE', 'LONGITUDE', 'CIDADE','LOGRADOURO']]

In [10]:
df = n_2016.head(5000)
# df = n_2016
wgs84_to_web_mercator2(df)

/Users/thiagolima/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/thiagolima/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [11]:
df['CRIME'] = df['RUBRICA'].apply(crime_classification)

/Users/thiagolima/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
source = ColumnDataSource(df)

In [13]:
# Cria dados para mostrar no hover
tooltips = [
    ('Ano', '@ANO'), 
    ('Mês','@MES'), 
    ('Crime', '@RUBRICA'),
    ('Cidade','@CIDADE'),
    ('Logradouro', '@LOGRADOURO')
]

In [14]:
p = figure(tools="pan,wheel_zoom,box_zoom,reset", x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator", plot_width=960, tooltips=tooltips)

p.add_tile(WMTSTileSource(url=url, attribution=attribution))

# p.circle('x','y', fill_color='Dodgerblue', size=10, fill_alpha=0.5, source=source, hover_fill_color="firebrick")
p.circle('x','y', fill_color='CRIME', size=8, fill_alpha=0.8, source=source)

show(p)